In [32]:
import numpy as np
import pandas as pd
import scipy.stats as stats
%matplotlib inline
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics # f1score, confusion_matrix, classification_report etc.
from __future__ import print_function
from sklearn.model_selection import train_test_split


In [2]:
!ls data

sample_submission.csv test.csv              train.csv


In [100]:
trainData = pd.read_csv("data/train.csv") # Add "../" on Kaggle

In [101]:
trainData.dtypes

id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object

In [102]:
trainData['none'] = 1 - trainData[labels].max(axis=1)
trainData.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.898321
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,0.302226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [103]:
trainData.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [104]:
labelCols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
trainDataLabels = trainData[labelCols] 

In [105]:
trainDataLabels.dtypes

toxic            int64
severe_toxic     int64
obscene          int64
threat           int64
insult           int64
identity_hate    int64
dtype: object

In [106]:
trainDataLabels.shape

(159571, 6)

In [107]:
trainDocs, testDocs, trainLabels, testLabels = train_test_split(trainData.comment_text, trainDataLabels)

In [108]:
print(trainDocs.shape, testDocs.shape, trainLabels.shape, testLabels.shape)

(119678,) (39893,) (119678, 6) (39893, 6)


In [109]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

In [110]:
trainDocs[trainLabels.toxic==1].sample(5)

101347    Hans Adler is a Nazi!!!!! He kept discrimating...
31578     Martin Conway the Desperado\n\nYou are a sad l...
64707     I'm keepin' your poop in a jar\nTill the day y...
117969    history of north korean aggression to south ko...
96991     Special note, for a special someone \n\nFuck y...
Name: comment_text, dtype: object

Below did not work because the final fit doesn't take multiple targets
```
from sklearn.pipeline import Pipeline
toxicClf = Pipeline(?[('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])
toxicClf.fit(trainDocs, trainLabels.toxic)
```

In [117]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
from sklearn.feature_extraction.text import TfidfTransformer
tfidftx = TfidfTransformer()
# Changing naming pattern
trainDocsCounts = cv.fit_transform(trainDocs)
trainDocsTfidf = tfidftx.fit_transform(trainDocsCounts)
trainDocsTfidf.shape

(119678, 159755)

In [122]:
from sklearn.naive_bayes import MultinomialNB
# toxicClf = MultinomialNB().fit(trainDocsTfidf, trainLabels.toxic)
# multiclassClf = MultinomialNB().fit(trainDocsTfidf, trainLabels) # ValueError: bad input shape (119678, 6)
labelsClf = { label: MultinomialNB().fit(trainDocsTfidf, trainLabels[label]) for label in labelCols}

In [123]:
trainDocsTfidf.shape

(119678, 159755)

In [130]:
testdocs = ['Fucking pig', 'Love love love.']
testdocs_counts = cv.transform(testdocs)
#print(testdocs_counts)
testdocs_tfidf = tfidftx.transform(testdocs_counts)
#print(testdocs_tfidf.shape)
for label in labelCols:
    print(label, labelsClf[label].predict(testdocs_tfidf))

#toxicClf=labelsClf['toxic']
#print(type(toxicClf))
#print(toxicClf.class_count_, toxicClf.classes_)
#predicted = toxicClf.predict(testdoc_tfidf)
#predicted
#for doc, cat in zip(testdocs, predicted):
#    print('%r => %s' % (doc, cat))

toxic [1 0]
severe_toxic [0 0]
obscene [1 0]
threat [0 0]
insult [0 0]
identity_hate [0 0]


In [131]:
for label in labelCols:
    testdocs_counts = cv.transform(testDocs)
    testdocs_tfidf = tfidftx.transform(testdocs_counts)
    testPredicts = labelsClf[label].predict(testdocs_tfidf)
    print(label, np.mean(testPredicts==testLabels[label]))

toxic 0.919710224851
severe_toxic 0.989672373599
obscene 0.952422730805
threat 0.997067154639
insult 0.952723535457
identity_hate 0.991427067405


The accuracy result as executed at 5:50pm 2/15 are shown in the middle column.
The far right column shows the accuracies of Jeremy Howard's [NB-SVM kernel](https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline/notebook), modified to use 9% of the training data to fit in my computer. 

| label | accuracy | NB-SVM |
| ----- | -------- | ------ |
| toxic | 0.9197 | 0.9242 |
| severe_toxic | 0.9897 | 0.9881 |
| obscene | 0.9524 | 0.9492 |
| threat | 0.9971 | 0.9944 |
| insult | 0.9527 | 0.9518 |
| identity_hate | 0.9914 | 0.9893 |